<a href="https://colab.research.google.com/github/samuveljebakumar/fish/blob/main/merlin_app_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/all data set"

SAMPLE_RATE = 16000
DURATION = 3
SAMPLES = SAMPLE_RATE * DURATION

N_MELS = 128
N_FFT = 1024
HOP_LENGTH = 512


In [ ]:
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, mono=True)
    audio = librosa.util.normalize(audio)
    return audio

def split_audio(audio):
    clips = []
    for i in range(0, len(audio) - SAMPLES, SAMPLES):
        clips.append(audio[i:i+SAMPLES])
    return clips

def extract_mel(audio_clip):
    mel = librosa.feature.melspectrogram(
        y=audio_clip,
        sr=SAMPLE_RATE,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db


In [ ]:
X = []
y = []

for category_folder in os.listdir(DATASET_PATH):
    category_path = os.path.join(DATASET_PATH, category_folder)
    if not os.path.isdir(category_path):
        continue

    print(f"Processing category: {category_folder}") # Diagnostic print for main category (e.g., train/test)

    for species_folder in os.listdir(category_path):
        species_path = os.path.join(category_path, species_folder)
        if not os.path.isdir(species_path):
            continue

        print(f"  Processing bird species: {species_folder}") # Diagnostic print for species

        for audio_file_name in os.listdir(species_path):
            file_path = os.path.join(species_path, audio_file_name)
            # Skip if it's not a file (e.g., another subdirectory or a hidden file)
            if not os.path.isfile(file_path):
                continue

            try:
                audio = load_audio(file_path)
                clips = split_audio(audio)

                for clip in clips:
                    mel = extract_mel(clip)
                    X.append(mel)
                    y.append(species_folder) # Assign species_folder as the label
                print(f"    Processed file: {audio_file_name} - clips found: {len(clips)}") # Diagnostic print for file
            except Exception as e:
                print(f"    Error processing file {file_path}: {e}") # Added error handling for file processing

X = np.array(X)
X = X[..., np.newaxis]  # add channel

In [ ]:
print("Number of samples:", len(X))
print("Number of labels:", len(y))
print("First 10 labels:", y[:10])


In [ ]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42
)


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu',
                 input_shape=(128, X.shape[2], 1)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Reshape((model.output_shape[1],
                    model.output_shape[2] * model.output_shape[3])))

model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(len(encoder.classes_), activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    validation_data=(X_test, y_test)
)


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc * 100)


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Bidirectional, GlobalAveragePooling2D

model = Sequential()

# First CNN block
model.add(Conv2D(64, (3,3), activation='relu', input_shape=(N_MELS, X.shape[2], 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Second CNN block
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Third CNN block
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Reshape output for LSTM
# Calculate the output shape dynamically for the reshape layer
# The input to LSTM needs to be 3D (batch_size, timesteps, features)
# Current model.output_shape before reshape: (None, height, width, channels)
# We want (None, height, width * channels)
reshape_dim_1 = model.output_shape[1] # height (timesteps)
reshape_dim_2 = model.output_shape[2] * model.output_shape[3] # width * channels (features)
model.add(Reshape((reshape_dim_1, reshape_dim_2)))

# LSTM layer
model.add(Bidirectional(LSTM(128, return_sequences=True))) # Added Bidirectional LSTM and return_sequences
model.add(Dropout(0.4))

model.add(LSTM(64)) # Added a second LSTM layer
model.add(Dropout(0.4))

# Output layer
model.add(Dense(len(encoder.classes_), activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test)
)